# Step 1: Fetch Movie Data from API


In [19]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

In [20]:
base_url = "https://api.themoviedb.org/3/movie/"

headers = {"accept": "application/json", "Authorization": f"Bearer {ACCESS_TOKEN}"}

movie_ids = [
    0,
    299534,
    19995,
    140607,
    299536,
    597,
    135397,
    420818,
    24428,
    168259,
    99861,
    284054,
    12445,
    181808,
    330457,
    351286,
    109445,
    321612,
    260513,
]

movies = []

# Fetch movie data for each ID
for movie_id in movie_ids:
    url = f"{base_url}{movie_id}?append_to_response=credits"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        movies.append(response.json())
    else:
        print(f"Failed to fetch data for movie ID {movie_id}: {response.status_code}")

with open("movies_data.json", "w") as f:
    json.dump(movies, f, indent=4)
    
# Create dataframe
movies_df = pd.DataFrame(movies)
print(f"Fetched {len(movies_df)} movies successfully.")
# print(movies_df.info())

Failed to fetch data for movie ID 0: 404
Fetched 18 movies successfully.


# Step 2: Data Cleaning and Preprocessing


In [21]:
# Drop irrelevant columns
movies_df.drop(
    columns=[
        "adult",
        "imdb_id",
        "original_title",
        "video",
        "homepage",
        "backdrop_path",
    ],
    inplace=True,
)
# print(movies_df.info())

# Extract name from collection
movies_df["collection_name"] = movies_df["belongs_to_collection"].apply(
    lambda x: x["name"] if isinstance(x, dict) and "name" in x else None
)

# Extract genre names
movies_df["genres"] = movies_df["genres"].apply(
    lambda x: "|".join([genre["name"] for genre in x])
)

# Extract spoken languages
movies_df["spoken_languages"] = movies_df["spoken_languages"].apply(
    lambda x: "|".join([lang["english_name"] for lang in x])
)

# Extract production countries
movies_df["production_countries"] = movies_df["production_countries"].apply(
    lambda x: "|".join([country["name"] for country in x])
)

# Extract production companies
movies_df["production_companies"] = movies_df["production_companies"].apply(
    lambda x: "|".join([company["name"] for company in x])
)

# Extract cast, director, and their sizes
movies_df["cast"] = movies_df["credits"].apply(
    lambda x: "|".join([member["name"] for member in x["cast"]])
)
movies_df["cast_size"] = movies_df["credits"].apply(lambda x: len(x["cast"]))
movies_df["director"] = movies_df["credits"].apply(
    lambda x: "|".join(
        [member["name"] for member in x["crew"] if member["job"] == "Director"]
    )
)
movies_df["crew_size"] = movies_df["credits"].apply(lambda x: len(x["crew"]))
movies_df.drop(columns=["credits"], inplace=True)

# Convert column data types
movies_df["budget"] = pd.to_numeric(movies_df["budget"], errors="coerce")
movies_df["id"] = pd.to_numeric(movies_df["id"], errors="coerce")
movies_df["popularity"] = pd.to_numeric(movies_df["popularity"], errors="coerce")
movies_df["revenue"] = pd.to_numeric(movies_df["revenue"], errors="coerce")
movies_df["runtime"] = pd.to_numeric(movies_df["runtime"], errors="coerce")
movies_df["release_date"] = pd.to_datetime(movies_df["release_date"], errors="coerce")

# Replace unrealistic values with NaN
movies_df.loc[movies_df["budget"] <= 0, "budget"] = pd.NA
movies_df.loc[movies_df["revenue"] <= 0, "revenue"] = pd.NA
movies_df.loc[movies_df["runtime"] <= 0, "runtime"] = pd.NA

# convert budget and revenue to millions
movies_df["budget_musd"] = movies_df["budget"] / 1_000_000
movies_df["revenue_musd"] = movies_df["revenue"] / 1_000_000
movies_df.drop(columns=["budget", "revenue"], inplace=True)
movies_df.loc[movies_df["vote_count"] == 0, "vote_average"] = pd.NA

# replace text placeholders with NaN
known_placeholders = ["", "-", "N/A", "No Data", "No overview", "Unknown", "None"]
movies_df["overview"] = movies_df["overview"].replace(known_placeholders, pd.NA)
movies_df["tagline"] = movies_df["tagline"].replace(known_placeholders, pd.NA)

# remove duplicates
movies_df.drop_duplicates(subset=["id"], keep="first", inplace=True)

# drop unknown id or title
movies_df.dropna(subset=["id", "title"], inplace=True)

# keep row with at least 10 non-NaN values
movies_df = movies_df[movies_df.notna().sum(axis=1) >= 10]

# filter only movies released then drop status
movies_df = movies_df[movies_df["status"] == "Released"]
movies_df.drop(columns=["status"], inplace=True)

ordered_columns = [
    "id",
    "title",
    "tagline",
    "release_date",
    "genres",
    "belongs_to_collection",
    "original_language",
    "budget_musd",
    "revenue_musd",
    "production_companies",
    "production_countries",
    "vote_count",
    "vote_average",
    "popularity",
    "runtime",
    "overview",
    "spoken_languages",
    "poster_path",
    "cast",
    "cast_size",
    "director",
    "crew_size",
]

movies_df = movies_df[ordered_columns]
movies_df.reset_index(drop=True, inplace=True)
movies_df.to_csv("cleaned_movies_data.csv", index=False)
movies_df.head(1)


,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,...,vote_average,popularity,runtime,overview,spoken_languages,poster_path,cast,cast_size,director,crew_size
0,299534,Avengers: Endgame,Avenge the fallen.,2019-04-24,Adventure|Science Fiction|Action,"{'id': 86311, 'name': 'The Avengers Collection...",en,356.0,2799.4391,Marvel Studios,...,8.238,13.3126,181.0,After the devastating events of Avengers: Infi...,English|Japanese|Xhosa,/bR8ISy1O9XQxqiy0fQFw2BX72RQ.jpg,Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chr...,105,Anthony Russo|Joe Russo,607


# Step 3: KPI Implementation & Analysis


In [24]:
# ranking function
def rank_movies(df, metric, ascending=False, number_of_results=10, min_budget=None, min_votes=None):

    data = df.copy()
    
    if min_budget is not None:
        data = data[data['budget_musd'] >= min_budget]
    if min_votes is not None:
        data = data[data['vote_count'] >= min_votes]
    
    # derived metrics 
    if metric == 'profit':
        data['profit'] = data['revenue_musd'] - data['budget_musd']
    elif metric == 'ROI':
        data['ROI'] = data['revenue_musd'] / data['budget_musd']
        
    ranked = data.sort_values(by=metric, ascending=ascending)
    
    return ranked.head(number_of_results)

# Highest revenue
ranked_by_revenue = rank_movies(movies_df, metric='revenue_musd', ascending=False)
print("HIGHEST REVENUE", ranked_by_revenue[['title', 'revenue_musd']])

# Highest budget
ranked_by_budget = rank_movies(movies_df, metric='budget_musd', ascending=False)
print("HIGHEST BUDGET", ranked_by_budget[['title', 'budget_musd']])

# Highest profit
ranked_by_profit = rank_movies(movies_df, metric='profit', ascending=False)
print("HIGHEST PROFIT", ranked_by_profit[['title', 'profit']])

# Lowest profit
ranked_by_lowest_profit = rank_movies(movies_df, metric='profit', ascending=True)
print("LOWEST PROFIT", ranked_by_lowest_profit[['title', 'profit']])

# Highest ROI
ranked_by_roi = rank_movies(movies_df, metric='ROI', ascending=False, min_budget=10_000_000)
print("HIGHEST ROI", ranked_by_roi[['title', 'ROI']])

# Lowest ROI
ranked_by_lowest_roi = rank_movies(movies_df, metric='ROI', ascending=True, min_budget=10_000_000)
print("LOWEST ROI", ranked_by_lowest_roi[['title', 'ROI']])

# Most voted movies
ranked_by_votes = rank_movies(movies_df, metric='vote_count', ascending=False)
print("MOST VOTED", ranked_by_votes[['title', 'vote_count']])

# Highest rated movies
ranked_by_rating = rank_movies(movies_df, metric='vote_average', ascending=False, min_votes=10)
print("HIGHEST RATED", ranked_by_rating[['title', 'vote_average']])

# Lowest rated movies
ranked_by_lowest_rating = rank_movies(movies_df, metric='vote_average', ascending=True, min_votes=10)
print("LOWEST RATED", ranked_by_lowest_rating[['title', 'vote_average']])

# Most popular movies
ranked_by_popularity = rank_movies(movies_df, metric='popularity', ascending=False)
print("MOST POPULAR", ranked_by_popularity[['title', 'popularity']])

    


HIGHEST REVENUE                            title  revenue_musd
1                         Avatar   2923.706026
0              Avengers: Endgame   2799.439100
4                        Titanic   2264.162353
2   Star Wars: The Force Awakens   2068.223624
3         Avengers: Infinity War   2052.415039
5                 Jurassic World   1671.537444
6                  The Lion King   1662.020819
7                   The Avengers   1518.815515
8                      Furious 7   1515.400000
13                     Frozen II   1453.683476
HIGHEST BUDGET                            title  budget_musd
0              Avengers: Endgame        356.0
3         Avengers: Infinity War        300.0
12      Star Wars: The Last Jedi        300.0
6                  The Lion King        260.0
2   Star Wars: The Force Awakens        245.0
1                         Avatar        237.0
9        Avengers: Age of Ultron        235.0
7                   The Avengers        220.0
10                 Black Panther      